In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [17]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [18]:
df = fetch_hourly_rides(12)

2025-03-06 11:23:07,410 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 11:23:07,417 INFO: Initializing external client
2025-03-06 11:23:07,417 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 11:23:08,105 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214708
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.07s) 


In [19]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-06 03:00:00+00:00,159,0
1,2025-03-06 03:00:00+00:00,162,8
2,2025-03-06 02:00:00+00:00,145,0
3,2025-03-06 02:00:00+00:00,69,0
4,2025-03-06 06:00:00+00:00,158,17
...,...,...,...
2263,2025-03-06 01:00:00+00:00,51,0
2264,2025-03-06 05:00:00+00:00,159,0
2265,2025-03-06 00:00:00+00:00,218,0
2266,2025-03-06 06:00:00+00:00,134,1


In [20]:
df_pred = fetch_predictions(12)

2025-03-06 11:23:15,352 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 11:23:15,360 INFO: Initializing external client
2025-03-06 11:23:15,361 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 11:23:16,080 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214708
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.04s) 


In [21]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,212,0.0,2025-03-06 08:00:00+00:00
1,52,0.0,2025-03-06 08:00:00+00:00
2,129,1.0,2025-03-06 08:00:00+00:00
3,145,-0.0,2025-03-06 08:00:00+00:00
4,140,51.0,2025-03-06 08:00:00+00:00
...,...,...,...
499,16,0.0,2025-03-06 17:00:00+00:00
500,174,0.0,2025-03-06 17:00:00+00:00
501,258,0.0,2025-03-06 17:00:00+00:00
502,131,0.0,2025-03-06 17:00:00+00:00


In [22]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [23]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand


In [24]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [25]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference


In [26]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference


In [27]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [28]:
mae_by_hour["MAE"].mean()

nan